In [1]:
import sys

if (path := "C:/Users/Tom/PycharmProjects/short_circuit") not in sys.path:
    sys.path.append(path)

# Three-Phase Near-To-Generator Short-Circuit
---

*This notebook solves example 4.6.1 from Schlabbach, J. (2005). Short-circuit Currents. IET.*

Consider the line diagram of a 220-kV system below.

<img src="./figures/ex1_line_diagram.png" style="max-width: 75%; height: auto; margin-left:auto; margin-right:auto">

For a three-phase short circuit at bus bar B, the branch short-circuit currents of the generators and the system feeder are to be calculated. Data of equipment is taken from the nameplates:

**Generator G1**
- $S_{rG1}$ = 120 MVA
- $U_{rG1}$ = 10.5 kV
- $\cos \varphi_{rG1}$ = 0.8
- $x''_{dG1}$ = 18%

**Generator G2**
- $S_{rG2}$ = 80 MVA
- $U_{rG2}$ = 10.5 kV
- $\cos \varphi_{rG2}$ = 0.85
- $x''_{dG2}$ = 16%

**Transformer T1**
- $S_{rT1}$ = 120 MVA
- $U_{rT1HV}/U_{rT1LV}$ = 220 kV/10.5 kV
- $u_{krT1}$ = 14%

**Transformer T2**
- $S_{rT2}$ = 80 MVA
- $U_{rT2HV}/U_{rT2LV}$ = 220 kV/10.5 kV
- $u_{krT2}$ = 12%

**Transformer T3**
- $S_{rT3}$ = 200 MVA
- $U_{rT3HV}/U_{rT3LV}$ = 400 kV/220 kV
- $u_{krT3}$ = 12%

**System feeder Q**
- $S''_{kQ}$ = 5 GVA
- $U_{nQ}$ = 380 kV

**Transmission lines L1 and L2**
- $X'_{L1}$ = $X'_{L2}$ = 0.4 ${\Omega}$/km
- $l$ = 50 km

## Impedance of Electrical Equipment 

In [2]:
from short_circuit import Quantity
from short_circuit.equipment import Generator, Transformer, PowerGrid

Q_ = Quantity

From the name plate data, the impedances of the network components can be calculated.

> **Voltage factor $c$ according to IEC 60909-0**<br>
The voltage factor $c$ takes account of the differences between the voltage at the short-circuit location and the internal voltage of system feeders, motors and generators due to voltage variations (time and place), operating of transformer tap-changer, etc.
> | Nominal system voltage $U_n$ | Maximal s.c. current $c_{max}$ | Minimal s.c. current $c_{min}$ |
> | ---------------------------- |:------------------------------:|:------------------------------:|
> | LV: 100 V up to 1000 V (inclusive), voltage tolerance +6% | 1.05 | 0.95 |
> | LV: 100 V up to 1000 V (inclusive), voltage tolerance +10% | 1.10 | 0.95 |
> | MV: > 1 kV up to 35 kV (inclusive) | 1.10 | 1.00 |
> | HV: > 35 kV | 1.10 | 1.00 |

**Impedance of system feeder**

In [3]:
feeder = PowerGrid(
    line_voltage=Q_(380, 'kV'),
    short_circuit_power=Q_(5, 'GVA'),
    R_to_X_ratio=1.e-9,
    voltage_factor=1.1
)

**Impedance of synchronous generator G1**

In [4]:
generator_G1 = Generator(
    nominal_voltage=Q_(10.5, 'kV'),
    nominal_power=Q_(120, 'MVA'),
    per_unit_reactance=Q_(18, 'pct'),
    power_factor=0.8,
    R_to_X_ratio=0.05,
    voltage_factor=1.1
)

**Impedance of synchronous generator G2**

In [5]:
generator_G2 = Generator(
    nominal_voltage=Q_(10.5, 'kV'),
    nominal_power=Q_(80, 'MVA'),
    per_unit_reactance=Q_(16, 'pct'),
    power_factor=0.85,
    R_to_X_ratio=0.05,
    voltage_factor=1.1
)

**Impedance of transformer T1 referred to secondary side**

In [6]:
transformer_T1 = Transformer(
    nominal_voltage=Q_(10.5, 'kV'),
    nominal_power=Q_(120, 'MVA'),
    percent_short_circuit_voltage=Q_(14, 'pct'),
    voltage_factor=1.1
)

**Impedance of transformer T2 referred to secondary side**

In [7]:
transformer_T2 = Transformer(
    nominal_voltage=Q_(10.5, 'kV'),
    nominal_power=Q_(80, 'MVA'),
    percent_short_circuit_voltage=Q_(12, 'pct'),
    voltage_factor=1.1
)

**Impedance of transformer T3 referred to secondary side**

In [8]:
transformer_T3 = Transformer(
    nominal_voltage=Q_(220, 'kV'),  # referred to secondary side
    nominal_power=Q_(200, 'MVA'),
    percent_short_circuit_voltage=Q_(12, 'pct'),
    voltage_factor=1.1
)

## Per-Unit Impedance of Electrical Equipment

In [9]:
from short_circuit.network.per_unit import PerUnitSystem

As indicated in the line diagram above, the system has three voltage levels (380 kV, 220 kV, and 10.5 kV) separated by transformers. To each voltage level in the system a per-unit system can be assigned based on the transformer turns ratio. First, a system-wide base MVA is chosen (100 MVA). The voltage bases for the different voltage levels in the system are chosen such that they match the transformer's voltage ratings (or turns ratio). For example, if a transformer has a 10:1 turns ratio, the base voltages on its primary and secondary sides are selected in a 10:1 ratio. When base voltages are selected this way, the per-unit impedance of the transformer becomes independent of the side being analyzed. That is, the impedance referred to the primary side is numerically equal to the impedance referred to the secondary side in per-unit terms.

In [10]:
volt_level_1 = PerUnitSystem(S_base=Q_(100, 'MVA'), U_base=Q_(380, 'kV'))
volt_level_2 = PerUnitSystem(S_base=Q_(100, 'MVA'), U_base=Q_(220, 'kV'))
volt_level_3 = PerUnitSystem(S_base=Q_(100, 'MVA'), U_base=Q_(10.5, 'kV'))

After the per-unit system of each voltage level in the system has been defined, the impedance of equipment is converted to per-unit value.

**At voltage level 1 (380 kV)**
- system feeder

In [11]:
Z_Q_pu = volt_level_1.get_per_unit_impedance(feeder.Z1)

**At voltage level 2 (220 kV)**
- transformer T3 (transformer impedance was calculated referred to secondary side, 220 kV)
- transmission lines L1 and L2

In [12]:
Z_T3_pu = volt_level_2.get_per_unit_impedance(transformer_T3.Z1)

The impedance of the transmission lines haven't been calculated yet.

In [13]:
Z_L1 = Q_(0.4j, 'ohm / km') * Q_(50, 'km')
Z_L2 = Q_(0.4j, 'ohm / km') * Q_(50, 'km')

Now, we can get the per-unit impedance of the transmission lines:

In [14]:
Z_L1_pu = volt_level_2.get_per_unit_impedance(Z_L1)
Z_L2_pu = volt_level_2.get_per_unit_impedance(Z_L2)

**At voltage level 3 (10.5 kV)**
- transformers T1 and T2 (their impedance was calculated referred to secondary side, 10.5 kV)
- synchronous generators G1 and G2

In [15]:
Z_T1_pu = volt_level_3.get_per_unit_impedance(transformer_T1.Z1)
Z_T2_pu = volt_level_3.get_per_unit_impedance(transformer_T2.Z1)
Z_G1_pu = volt_level_3.get_per_unit_impedance(generator_G1.Z1)
Z_G2_pu = volt_level_3.get_per_unit_impedance(generator_G2.Z1)

## Impedance Matrix

In [16]:
from short_circuit.network import Network

After the per-unit impedances of all network components in the power system have been determined, the network can be build up from the ground (or reference node, 0 V). This is done branch by branch. A branch in a network runs between two network nodes: a start and an end node. Besides the reference node (ground), in the power system of our example there are three other nodes: node (bus) 'Q', node (bus) 'A', and node (bus) 'B' (see the line diagram above). **Do note that branches connected at one end to the reference node, must always have the reference node as their start node.** If a branch contains a voltage source, this must be indicated by setting parameter `has_source` to `True`.

The power network in our example has the following branches:
1. branch from the reference node (ground) to node 'Q', containing the per-unit impedance of the system feeder;
2. branch from the reference node (ground) to node 'B', containing the per-unit impedance of generator G1 and the per-unit impedance of transformer T1 connected in series;
3. branch from the reference node (ground) to node 'B', containing the per-unit impedance of generator G2 and the per-unit impedance of transformer T2 connected in series;
4. branch from node 'Q' to node 'A', containing the per-unit impedance of transformer T3;
5. branch from node 'A' to node 'B', containing the per-unit impedance of transmission line L1;
6. branch from node 'A' to node 'B', containing the per-unit impedance of transmission line L2.

In [17]:
network = Network()
network.add_branch(Z_Q_pu, None, 'Q', has_source=True)
network.add_branch(Z_T3_pu, 'Q', 'A')
network.add_branch(Z_L1_pu, 'A', 'B')
network.add_branch(Z_L2_pu, 'A', 'B')
network.add_branch(Z_T1_pu + Z_G1_pu, None, 'B', has_source=True)
network.add_branch(Z_T2_pu + Z_G2_pu, None, 'B', has_source=True)

Once the network is ready, the impedance matrix is also determined.

In [18]:
network.show_impedance_matrix()

                    Q                   A                   B
Q  0.000033+0.020069j  0.000121+0.014937j  0.000152+0.013124j
A  0.000121+0.014937j  0.000443+0.054647j  0.000556+0.048014j
B  0.000152+0.013124j  0.000556+0.048014j  0.000699+0.060339j


## Initial Symmetrical Short-Circuit Currents

In [19]:
from short_circuit.faults.three_phase_fault import ThreePhaseFault

From the problem statement, a three-phase short circuit occurs at bus bar B (node 'B').

In [20]:
faulted_network = ThreePhaseFault(network, U_prefault=1.0, c=1.1)
faulted_network.set_faulted_node('B')

Parameter `U_phase` is the nominal system (line-to-ground) voltage at the fault location before the fault. As we are working with per-unit quantities, the system voltage must also be expressed in per-unit. The base voltages of the per-unit systems are chosen to be equal to the nominal system voltages in the power system, so the per-unit value is 1.0 pu. The fault current in the three-phase short-circuit at node 'B' can now be retrieved with:  

In [21]:
I_k3i_B_pu = faulted_network.get_fault_current()
I_k3i_B_pu

(0.21126094510185126-18.227977082124927j)

Note that is also a per-unit value, as the network has been created with the per-unit impedance of the branches. To get the actual short-circuit current, we need to multiply the per-unit current with the base current at the fault location, which belongs to voltage level 2 (220 kV). 

In [22]:
I_k3i_B = I_k3i_B_pu * volt_level_2.I_base
print(f"{abs(I_k3i_B.to('kA')):~P.4f}")

4.7839 kA


To determine the contributions of the system feeder and the generators to the short-circuit current at 'B', we need to know the IDs of the branches in the network.

In [23]:
for branch in network.branches:
    print(branch)

Branch 1 [ref->Q]: Z = (2.1999998282418335e-11+0.022000000000000002j)
Branch 2 [Q->A]: Z = (3.581406450864694e-18+0.058488805970149244j)
Branch 3 [A->B]: Z = 0.04132231404958678j
Branch 4 [A->B]: Z = 0.04132231404958678j
Branch 5 [ref->B]: Z = (0.007445848375451274+0.2613862172015222j)
Branch 6 [ref->B]: Z = (0.010144932067154035+0.34912065626845357j)


Branch 1 contains the system feeder, branch 5 contains generator G1, and branch 6 (i.e. the last branch that we added to the network) contains generator G2.

In [24]:
I_k3i_Q_pu = faulted_network.get_branch_current(("ref", "Q"))
I_k3i_G1_pu = faulted_network.get_branch_current(("ref", "B"))
I_k3i_G2_pu = faulted_network.get_branch_current(6)
I_k3i_Q = I_k3i_Q_pu * volt_level_1.I_base
I_k3i_G1 = I_k3i_G1_pu * volt_level_3.I_base
I_k3i_G2 = I_k3i_G2_pu * volt_level_3.I_base

In [25]:
from short_circuit import complex_quantity_to_polar_string

In [26]:
print(
    f"- short-circuit bus bar B: {complex_quantity_to_polar_string(I_k3i_B.to('kA'))}",
    f"- feeder Q: {complex_quantity_to_polar_string(I_k3i_Q.to('kA'))}",
    f"- generator G1: {complex_quantity_to_polar_string(I_k3i_G1.to('kA'))}",
    f"- generator G2: {complex_quantity_to_polar_string(I_k3i_G2.to('kA'))}",
    sep='\n'
)

- short-circuit bus bar B: 4.7839 kA < -89.34°
- feeder Q: 1.6523 kA < -90.00°
- generator G1: 23.1304 kA < -88.37°
- generator G2: 17.3174 kA < -88.34°


Note that transformers T1 and T2 will reduce the current going from the generators to node 'B' at their primary side, while transformer T3 will raise the current coming from the feeder on its secondary side. 

In [27]:
N_T1 = N_T2 = 220 / 10.5
N_T3 = 380 / 220
I_k3i_G1_pri = I_k3i_G1.to('kA') * (1 / N_T1)
I_k3i_G2_pri = I_k3i_G2.to('kA') * (1 / N_T2)
I_k3i_Q_sec = I_k3i_Q.to('kA') * N_T3

In [28]:
print(
    f"current from G1 at B: {complex_quantity_to_polar_string(I_k3i_G1_pri)}",
    f"current from G2 at B: {complex_quantity_to_polar_string(I_k3i_G2_pri)}",
    f"current from Q at B: {complex_quantity_to_polar_string(I_k3i_Q_sec)}",
    sep='\n'
)

current from G1 at B: 1.1040 kA < -88.37°
current from G2 at B: 0.8265 kA < -88.34°
current from Q at B: 2.8539 kA < -90.00°


It can be checked that the sum of these three currents equals short-circuit current `I_k3i_B` at node 'B'.